<a href="https://colab.research.google.com/github/fle1scha/DL4NTP/blob/main/DL4NTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import tensorflow and numpy
import tensorflow as tf
import numpy as np 
import pandas as pd

In [ ]:
#Import keras models
from random import random
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed

1. Mount Google Drive file system

In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')

2. Read in SANReN sample data.

In [ ]:
with open('/content/gdrive/My Drive/SANReN.txt') as f:
  SANReN = f.readlines()
  
#Iterate through first 10 rows to ensure data has been read correctly. 
'''for i in range(10):
  print(SANReN[i])'''

3. Clean dataframe headers

In [ ]:
headings_line = SANReN[0].split()
#Merge 'Src', 'IP', and 'Addr:Port' 
headings_line[4:7] = [''.join(headings_line[4:7])]
#Merge 'Dst', 'IP', and 'Addr:Port' 
headings_line[5:8] = [''.join(headings_line[5:8])]
#Remove 'Flags', 'Tos', and 'Flows'.
headings_line = headings_line[0:6] + headings_line[8:13]


4. Clean time-series data points

In [ ]:
framedata = []
for i in range(1, len(SANReN)):
  data_line = SANReN[i].split()

  if (data_line[11] == "M" and data_line[14] == 'M'): #Bytes and BPS in megabytes
    #print("1 and 2") 
    data_line = data_line[0:5] + data_line[6:7] + data_line[9:16]
    data_line[7:9] = [''.join(data_line[7:9])]
    data_line[9:11] = [''.join(data_line[9:11])]
  
  elif (data_line[13] == 'M'): #BPS measured in megabytes
    #print("2")
    data_line = data_line[0:5] + data_line[6:7] + data_line[9:15]
    data_line[9:11] = [''.join(data_line[9:11])]
    
  elif data_line[11] == 'M': #Bytes measured in megabytes
    #print("1")
    data_line = data_line[0:5] + data_line[6:7] + data_line[9:15]
    data_line[7:9] = [''.join(data_line[7:9])]
  
  else: #No megabyte metrics
    #print("0")
    data_line = data_line[0:5] + data_line[6:7] + data_line[9:14]
  
  framedata.append(data_line)
  

5. Convert numpy array into pandas dataframe for LSTM.

In [ ]:

df = pd.DataFrame(np.array(framedata), columns=headings_line)
print(df)

          Date    first-seen Duration Proto  ...   Bytes   pps     bps   Bpp
0   2020-07-04  20:10:06.480    1.223   TCP  ...  234000  3679    1.5M    52
1   2020-07-04  20:09:01.555   78.205   TCP  ...   46.5M   396    4.8M  1500
2   2020-07-04  20:10:01.690    5.307   TCP  ...   40000   188   60297    40
3   2020-07-04  20:09:23.019   43.982   TCP  ...    3.0M    45  545677  1500
4   2020-07-04  20:10:07.007    0.000   TCP  ...  750000     0       0  1500
..         ...           ...      ...   ...  ...     ...   ...     ...   ...
78  2020-07-04  20:10:07.454    0.000   TCP  ...  750000     0       0  1500
79  2020-07-04  20:10:07.458    0.000   TCP  ...  728000     0       0  1456
80  2020-07-04  20:10:07.463    0.000   TCP  ...   20000     0       0    40
81  2020-07-04  20:10:07.464    4.021   TCP  ...    5.2M   870   10.4M  1500
82  2020-07-04  20:10:07.467    0.000   TCP  ...   26000     0       0    52

[83 rows x 11 columns]


**LSTM IMPLEMENTATION WILL START HERE **

